In [3]:
from Google_business_search import get_google_places_by_location
from database import Database

db = Database()
coordinates = db.select_rows('''SELECT lat, lng FROM coordinates LIMIT 10''')
coordinates_list = [coord for coord in coordinates]
short_coordinates = [','.join(map(str, coordinate)) for coordinate in coordinates_list]

# result = get_google_places_by_location(short_coordinates[0])
# len(result)

In [1]:
import review_scraper

google_test_url = 'https://maps.google.com/?cid=4988939335835406517'
google_id = 'randomID8083rpoeu'
data = review_scraper.scrape_google_reviews(google_url=google_test_url, google_id=google_id)
print(len(data)) # should be 11

Scraped 11 reviews from google business id #randomID8083rpoeu
11


In [5]:
import requests, json, time
from Google_business_search import get_google_place_url_and_review_count

from dotenv import load_dotenv
import os

load_dotenv()
API_key = os.getenv('GOOGLE_API_KEY_2')

coordinates = '40.6038852,-74.0062078'
business_type='restaurant'
search_term='halal'
radius = '16093'
next_page=''


def get_google_places_by_location(coordinates, business_type='restaurant', search_term='halal', radius = '16093', next_page=''):
    URL = ('https://maps.googleapis.com/maps/api/place/textsearch/json?location='
        + coordinates + '&radius=' + radius + 'query=' + search_term + '&type='
        + business_type + '&pagetoken=' + next_page + '&key='+ API_key)
    response = requests.get(URL)
    json_obj = json.loads(response.text)
    results = json_obj["results"]

    # # SQL query to add business to table
    # db = Database()
    # update_sql = """INSERT INTO businesses (name, platform_id, url, total_review_count, address)
    #                 VALUES (%s, %s, %s, %s, %s)
    #                 ON CONFLICT (url) DO NOTHING"""
    businesses_list = []
    print(json_obj['status'])
    for result in results:
        name = result['name']
        google_id = result['place_id']
        google_url , review_count = get_google_place_url_and_review_count(google_id)
        address = result['formatted_address']
        businesses_list.append([name, google_id, google_url, review_count, address])
        # db.insert_row( update_sql, *(name, google_id, google_url, review_count, address))
    try:
        print('Found next page')
        next_page_token = json_obj["next_page_token"]
    except:
        #no next page
        
        return businesses_list
    time.sleep(1)
    data = get_google_places_by_location(coordinates=coordinates, next_page=next_page_token)
    businesses_list.extend(data)
    print('Found {} businesses near coordinates {}'.format(len(businesses_list), coordinates))
    return businesses_list

In [10]:
import sys
sys.exit

<function sys.exit>